In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split

testData = pd.read_csv("test.csv")
testY = pd.read_csv("output.csv")
testData['stroke'] = testY['stroke']
trainData = pd.read_csv("train.csv")

#Filling Missing BMI Data === train

temp = trainData['bmi'].mean() + trainData['bmi'].median()
temp /= 2
temp
trainData['bmi'] = trainData['bmi'].fillna(temp)

#Filling Missing BMI Data ===test

temp = testData['bmi'].mean() + testData['bmi'].median()
temp /= 2
temp
testData['bmi'] = testData['bmi'].fillna(temp)

print( "Empty Smoking Status(Train) in " , 
     len( list( filter( lambda x: x == True , pd.isna(trainData['smoking_status']) ) ) )
     )


print( "Empty Smoking Status(Test) in " , 
     len( list( filter( lambda x: x == True , pd.isna(testData['smoking_status']) ) ) )
     )

# Segregating into Two Models

trainDataNS = trainData[ trainData['smoking_status'].isna() ]
trainDataNS = trainDataNS.drop( columns=['smoking_status'] )
trainDataS = trainData[ trainData['smoking_status'].notna() ]

testDataNS = testData[ testData['smoking_status'].isna() ]
testDataNS = testDataNS.drop( columns=['smoking_status'] )
testDataS = testData[ testData['smoking_status'].notna() ]

print( "Imbalance in train is ", trainDataS['stroke'].value_counts() )
print( "Imbalance in test is ", testDataS['stroke'].value_counts() )

# X and Y division

yTrainDataNS = trainDataNS['stroke']
xTrainDataNS = trainDataNS.drop( columns=['stroke'] )

yTrainDataS = trainDataS['stroke']
xTrainDataS = trainDataS.drop( columns=['stroke'] )

yTestDataNS = testDataNS['stroke']
xTestDataNS = testDataNS.drop( columns=['stroke'] )

yTestDataS = testDataS['stroke']
xTestDataS = testDataS.drop( columns=['stroke'] )

#Nominal Categories to one-hot encoding === Train

#Smoke Data
xTrainDataS = pd.get_dummies( xTrainDataS, 
                             columns=['gender', 'ever_married', 'work_type' , 'Residence_type', 'smoking_status']
                            , prefix=['sex', 'married', 'work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTrainDataNS = pd.get_dummies( xTrainDataNS,
                              columns=['gender', 'ever_married', 'work_type' , 'Residence_type']
                            , prefix=['sex', 'married', 'work_type' , 'res_type'] )



#Nominal Categories to one-hot encoding ===Test

#Smoke Data
xTestDataS = pd.get_dummies( xTestDataS,
                            columns=['gender', 'ever_married', 'work_type' , 'Residence_type', 'smoking_status']
                            , prefix=['sex', 'married', 'work_type' , 'res_type', 'smoke'] )


#No Smoke Data
xTestDataNS = pd.get_dummies( xTestDataNS,
                             columns=['gender', 'ever_married', 'work_type' , 'Residence_type']
                            , prefix=['sex', 'married', 'work_type' , 'res_type'] )

Empty Smoking Status(Train) in  13292
Empty Smoking Status(Test) in  5751
Imbalance in train is  0    29470
1      638
Name: stroke, dtype: int64
Imbalance in test is  0    9443
1    3407
Name: stroke, dtype: int64


In [2]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
lr = LR()

In [19]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
#sm=SMOTE()
print("1st Check",Counter(yTrainDataNS))
print("1st Check",Counter(yTestDataNS))
#sm_x,sm_y=sm.fit_sample(xTrainDataNS,yTrainDataNS)
#print("1st Check",Counter(sm_y))
#x_train, x_test, y_train, y_test = train_test_split(sm_x,sm_y, test_size=0, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(xTestDataNS,yTestDataNS, test_size=0, random_state=0)


lr.fit(x_train, y_train)
clf.fit(x_train,y_train)

1st Check Counter({0: 13147, 1: 145})
1st Check Counter({0: 4207, 1: 1544})


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [22]:
y_test=yTrainDataNS
x_test=xTrainDataNS
predictions = lr.predict(x_test)
predictions_Tree=clf.predict(x_test)

print(lr.score(x_test, y_test))
print(clf.score(x_test, y_test))


0.9890159494432742
1.0


In [11]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, predictions))
print(metrics.confusion_matrix(y_test,predictions_Tree))
#MCC is better metric along with F-Score


[[3410  797]
 [1219  325]]
[[4160   47]
 [1518   26]]


In [12]:
#Logistic 
metrics.matthews_corrcoef(y_test, predictions, sample_weight=None)

0.023536443019994725

In [13]:
#Tree
metrics.matthews_corrcoef(y_test, predictions_Tree, sample_weight=None)

0.02243592501341416

In [14]:
#Logistic
metrics.f1_score(y_test, predictions, labels=None, pos_label=1, average='binary', sample_weight=None)

0.24381095273818457

In [15]:
#Tree
metrics.f1_score(y_test, predictions_Tree, labels=None, pos_label=1, average='binary', sample_weight=None)

0.032158317872603585

In [ ]:
testDataS.head()

In [ ]:
trainDataS.head()